# Question 2

To address "Blind Spots" (what happens before, between, or after the identified process steps in ITAM), add process steps based on the video data identified (e.g., "scene01_video_ocel.json"). Demonstrate how the processes now appear with these enriched data sets (i.e., events from ITAM and video data).


#### Import packages


In [1]:
import pm4py
import pandas as pd

In [ ]:
# Read ITAM XES file
itam_xes_path = "filtered_itam_xes.xes"
itam_xes = pm4py.read_xes(itam_xes_path)

def read_scene_xes(count: str, instance: str):
    """Liest eine einzelne Szenen-XES-Datei basierend auf der Nummer."""
    path = f"../dataProcessing/filteredSceneVideoXES/filtered_scene{count}_video_{instance}.xes"  
    return pm4py.read_xes(path)

def combine_itam_scenes(scene_counts: list, instance: str):
    combined_df = pd.DataFrame(itam_xes)
    for count in scene_counts:
        # Szenen-XES-Datei lesen und hinzufügen
        scene_df = pd.DataFrame(read_scene_xes(count, instance))
        combined_df = pd.concat([combined_df, scene_df], ignore_index=True)
    # Nach Zeitstempel sortieren
    combined_df.sort_values(by='time:timestamp', inplace=True)
    return combined_df

def write_combined_xes(scene_counts: list, instance: str):
    combined_log = combine_itam_scenes(scene_counts, instance)
    output_path = f"../dataProcessing/itamSceneCombinedXES/itam_scene_combined_{instance}.xes"
    pm4py.write_xes(combined_log, output_path)
    print(f"Kombinierte XES-Datei gespeichert unter: {output_path}")

In [ ]:
# Write combined file

# Asset disbursement to client
write_combined_xes(['01','07','09','12','13','15'], 'adtc')

# New inventory asset
write_combined_xes(['02','03','04','06','07','10'], 'nai')

# Self-service asset check-out
write_combined_xes(['03','05','06','11','14'], 'ssaco')

## Visualize Processes


In [ ]:
import pm4py

### XES

# New inventory asset
xes_nai = pm4py.read_xes('../dataProcessing/itamSceneCombinedXES/itam_scene_combined_nai.xes')

dfg, start_activities, end_activities = pm4py.discover_dfg(xes_nai, 
                                                           case_id_key='case:concept:name',
                                                           activity_key='concept:name',
                                                           timestamp_key='time:timestamp'
                                                           )
pm4py.vis.view_dfg(dfg, start_activities, end_activities)

bpmn = pm4py.discover_bpmn_inductive(xes_nai)
pm4py.vis.view_bpmn(bpmn)

In [ ]:
# Asset disbursement to client
xes_adtc = pm4py.read_xes('../dataProcessing/itamSceneCombinedXES/itam_scene_combined_adtc.xes')

dfg, start_activities, end_activities = pm4py.discover_dfg(xes_adtc, 
                                                           case_id_key='case:concept:name',
                                                           activity_key='concept:name',
                                                           timestamp_key='time:timestamp'
                                                           )
pm4py.vis.view_dfg(dfg, start_activities, end_activities)

bpmn = pm4py.discover_bpmn_inductive(xes_adtc)
pm4py.vis.view_bpmn(bpmn)

In [ ]:
# Self-service asset check-out
xes_ssaco = pm4py.read_xes('../dataProcessing/itamSceneCombinedXES/itam_scene_combined_ssaco.xes')

dfg, start_activities, end_activities = pm4py.discover_dfg(xes_ssaco, 
                                                           case_id_key='case:concept:name',
                                                           activity_key='concept:name',
                                                           timestamp_key='time:timestamp'
                                                           )
pm4py.vis.view_dfg(dfg, start_activities, end_activities)

bpmn = pm4py.discover_bpmn_inductive(xes_ssaco)
pm4py.vis.view_bpmn(bpmn)

### Visualize Filtered ITAM


In [ ]:
import pm4py
### filter itam_ocel

### OCEL
itam_ocel = pm4py.read_ocel_json('../global/itam_ocel.jsonl')
filtered_itam_ocel = pm4py.filtering.filter_ocel_events_timestamp(itam_ocel,min_timest='2024-03-25 08:48:23', max_timest='2024-03-25 12:02:12')
filtered_itam_ocel_path = '../dataProcessing/filtered_itam_ocel'
pm4py.write_ocel_json(filtered_itam_ocel,filtered_itam_ocel_path)

vis = pm4py.discover_ocdfg(filtered_itam_ocel)
pm4py.vis.view_ocdfg(vis)

### XES
itam_xes = pm4py.read_xes('../dataProcessing/itam_xes.xes')
filtered_itam_xes = pm4py.filtering.filter_time_range(itam_xes, dt1='2024-03-25 08:48:23', dt2='2024-03-25 12:02:12', mode = 'events', case_id_key='case:concept:name', timestamp_key='time:timestamp')
filtered_itam_xes_path = '../dataProcessing/filtered_itam_xes.xes'
pm4py.write_xes(filtered_itam_xes,filtered_itam_xes_path)

dfg, start_activities, end_activities = pm4py.discovery.discover_dfg(filtered_itam_xes,
                                                                     case_id_key='case:concept:name',
                                                                     activity_key='concept:name',
                                                                     timestamp_key='time:timestamp')
pm4py.vis.view_dfg(dfg, start_activities, end_activities)